## Overview

It has been a little bit since my last entry, but I have been traveling. On my travels I realized both how much and how little power that I need in order to be self sustaining in my van. With that said, I want to create a tracker to aggregate my total power consumption based on usage and time. It will also feature a way to convert watts to amps and amps to watts for each entry. The caveat is that I want to create functions for almost every interaction that the user (me) will have.

## Code

In [251]:
# Importing the csv library to write to later

In [252]:
import csv 

In [253]:
# Create conversion functions from watts to amps and amps to watts
def convert_to_amps(wattage, voltage):
    return float( wattage / voltage)

def convert_to_watts(amps, voltage):
    return float(amps * voltage)

In [254]:
# Create a function that takes in input
def user_logging():
    objectName = input('What is the name of the item?').upper()

    powerType = input('Is consumption in amps (A) or watts (W)?').upper()
    
    while powerType.upper() != 'A' and powerType != 'W':
        powerType = input('Please enter A for amps or W for wattage').upper()

    voltage = input('Is the voltage 12 or 110?')

    while voltage not in ['12','110']:
         voltage = input('Please enter a voltage of 12 or 110?')

    voltage = int(voltage)

    consumption = float(input('What is the consumption per hour?'))

    usageTime = float(input('How much time in minutes are you planning on using the item?'))

    item_dict[objectName] = [powerType, voltage, consumption, usageTime]

    return objectName,powerType,voltage,consumption,usageTime

In [255]:
# Create function to convert and total the power consumption

def convert_and_total(power,voltage,consumption,time):
    
    if power == 'A':
        consumptionWattage = convert_to_watts(consumption,voltage)
    else:
        consumptionWattage = consumption

    total_consumption = consumptionWattage * time/60

    total_amperage = convert_to_amps(consumptionWattage,voltage)

    return consumptionWattage,total_consumption,total_amperage

In [256]:
# Create a function that calculates % of power bank and charge time

def power_draw_calculator(consumption):
    percentage = float(consumption / 2200)
    chargeTime = round((consumption / 220) * 60,1)
    
    return percentage, chargeTime

In [257]:
# Putting all the functions together

def workflow_():
    objectName, powerType,voltage,consumption,usageTime = user_logging()
    consumptionWattage,total_consumption,total_amperage = convert_and_total(powerType,voltage,consumption,usageTime)
    percentage, chargeTime = power_draw_calculator(total_consumption)

    item_dict[objectName] = [powerType, voltage, usageTime, consumptionWattage, total_consumption, round(total_amperage,1), round(percentage,3), round(chargeTime,3)]

In [258]:
def start():
    continueVar = str()

    while continueVar.upper() != 'N':
        workflow_()
        
        continueVar = input('Do you want to add another item Y/N?').upper()

        while continueVar.upper() not in ['Y','N']:
            continueVar = input('Please enter Y/N?').upper()

In [259]:
# Open a file and check the for existing items from the dictionary. If items exist remove the values from the dictionary
# to prepare for processing

def check_file(dict):

    decisionVar = str()
    
    with open('/Users/MarkHinojosa/Python_Learn/Files/power_consumption.csv','r') as file:
        reader = csv.DictReader(file)
    
        for row in reader:
            if row['item'] in dict.keys():
                decisionVar = input(f"{row['item']} already exists do you want to add a new line? Y/N?")

                while decisionVar.upper() not in ['Y','N']:
                    decisionVar = input(f"Please enter Y/N to add new line for {row['item']}")

                if decisionVar.upper() == 'N':
                    dict.pop(row['item'])
                elif decisionVar.upper() == 'Y':
                    pass

In [260]:
# Unpack the dictionary into a nested list to unpack later for csv writing

def prepare_for_csv(dict):
    csv_list = str()
    value_list = []
    
    for keys in item_dict.keys():
        value_list.append(keys)
            
        for value in item_dict[keys]:
            value_list.append(value)

    return value_list

In [261]:
# Split a part the list into a nested list for each key from the dictionary

def list_split(list):
    final_list = []
    idx_counter = 0
    start_position, end_position = 0,9
    
    while idx_counter < int(len(list) / 9):
    
        final_list.append(list[start_position:end_position])
    
        idx_counter += 1
        start_position += 9
        end_position += 9

    return final_list

In [262]:
# Enumerate through list and add to file before closing

def write_lines(list):
    
    with open('/Users/MarkHinojosa/Python_Learn/Files/power_consumption.csv','a') as file:
        writer = csv.writer(file)

        for idx, value in enumerate(list):
            writer.writerow(list[idx])

In [263]:
# Create a function to return the total power consumption and charge time needed

def power_and_time():

    total_power = float()
    total_time =float()
    power_percentage = float()
    record_count = 0
    
    with open('/Users/MarkHinojosa/Python_Learn/Files/power_consumption.csv','r') as file:
        reader = csv.DictReader(file)

        for row in reader:
            total_power = total_power + float(row['total_consumption'])
            record_count += 1


    power_percentage = round(total_power / 2200,3)

    return total_power,power_percentage,record_count

In [264]:
# Sets the starting dictionary to a blank value and estimated charging time
# Starts the workflow by asking if an entry should be added or not

item_dict = {}
daily_charge = 220 * 8

start_var = input('Do you want to add a new entry? Y/N')

while start_var.upper() not in ['Y','N']:
    start_var = input('Please enter a Y/N')

if start_var.upper() == 'Y':
    start()
    check_file(item_dict)
    write_lines(list_split(prepare_for_csv(item_dict)))
    print(f'{len(item_dict)} items were added to the list')

power,percentage,records = power_and_time()
print(f'''
There are {records} entries in the power journal. 
The total consumed wattage is {power} which is {percentage * 100} of available power.
Based on average solar input, it will take {int(round((power / 220) * 60,0))} minutes to charge.
In optimal conditions, the daily expected charge is {daily_charge} watts, which is a difference of {power - daily_charge}.
The charge time for the remainder is {int(round(((power - daily_charge) / 220) * 60,0))} minutes.''')

Do you want to add a new entry? Y/N Y
What is the name of the item? PC
Is consumption in amps (A) or watts (W)? w
Is the voltage 12 or 110? 110
What is the consumption per hour? 330
How much time in minutes are you planning on using the item? 60
Do you want to add another item Y/N? n


1 items were added to the list

There are 9 entries in the power journal. 
The total consumed wattage is 1376.0 which is 62.5 of available power.
Based on average solar input, it will take 375 minutes to charge.
In optimal conditions, the daily expected charge is 1760 watts, which is a difference of -384.0.
The charge time for the remaainder is -105 minutes.


## Conclusion

This was such an ambitious project but completing it gave me more confidence to at least try amibtious projects in the future. There were many moments where I had to stop to really think about the outcome and looking back already, I think I could have been more efficient. Here is what a I learned from this project:

- How to fully take advantage of 'not in' to in order to force the correct type of input from the user
- How to open files to check for an existing index
- How to move dictionar keys and values into a list as a single entry
- How to write into a file from a list
- How to next functions to get a desired output